<a href="https://colab.research.google.com/github/nievesyu/big-data-challenge/blob/main/BigData_Amazon_Major_Appliances.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/graph

In [ ]:
# For connection to Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-10-02 21:31:10--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.24MB/s    in 0.7s    

2021-10-02 21:31:11 (1.24 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data
# Instructions mention that Tab ('\t')
from pyspark import SparkFiles
url='https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Major_Appliances_v1_00.tsv.gz'
spark.sparkContext.addFile(url)
review_df = spark.read.csv(SparkFiles.get('amazon_reviews_us_Major_Appliances_v1_00.tsv.gz'), sep='\t', header=True, inferSchema=True)

# Show DataFrame
review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16199106|R203HPW78Z7N4K|B0067WNSZY|     633038551|FGGF3032MW Galler...|Major Appliances|          5|            0|          0|   N|                Y|If you need a new...|What a great stov...| 2015-08-31|
|         US|   16374060|R2EAIGVLEALSP3|B002QSXK60|     811766671|Best Hand Clothes...|Major Appliances|          5|    

In [ ]:
# Count
print(review_df.count())

96901


In [ ]:
# Drop null values
dropna_df = review_df.dropna()
dropna_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16199106|R203HPW78Z7N4K|B0067WNSZY|     633038551|FGGF3032MW Galler...|Major Appliances|          5|            0|          0|   N|                Y|If you need a new...|What a great stov...| 2015-08-31|
|         US|   16374060|R2EAIGVLEALSP3|B002QSXK60|     811766671|Best Hand Clothes...|Major Appliances|          5|    

In [ ]:
# Count
print(dropna_df.count())

96888


In [ ]:
from pyspark.sql.functions import col

In [ ]:

review_id_table_df=dropna_df.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date'])
review_id_table_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R203HPW78Z7N4K|   16199106|B0067WNSZY|     633038551| 2015-08-31|
|R2EAIGVLEALSP3|   16374060|B002QSXK60|     811766671| 2015-08-31|
|R1K1CD73HHLILA|   15322085|B00EC452R6|     345562728| 2015-08-31|
|R2KZBMOFRMYOPO|   32004835|B00MVVIF2G|     563052763| 2015-08-31|
| R6BIZOZY6UD01|   25414497|B00IY7BNUW|     874236579| 2015-08-31|
|R1MCXZFNF8E7Y0|   36311751|B0033X29CI|     294467812| 2015-08-31|
|R3EMB3E3ODR6BW|   30920961|B005R597HA|     183784715| 2015-08-31|
| RJTONVTTOPJ5S|   52491265|B00MO6V8Y0|     960251524| 2015-08-31|
|R21U5QZ2CQECUM|   48166169|B00HT39QDI|     992475314| 2015-08-31|
| RL2BBC51H89DH|   50394924|B00LESFZ52|       1641606| 2015-08-31|
|R3RNEPHF3WIRSZ|    3915552|B0149IJVPI|     838108342| 2015-08-31|
|R38DNT9KML2PF3|   17068589|B002HT0958|     387104338| 2015-08

In [ ]:
products_df=dropna_df.select(['product_id', 'product_title']).distinct()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00RE645WC|WindMax 23" Stain...|
|B008SCVL6E|G.E. Microwave Gl...|
|B0081LV860|Samsung WF405ATPA...|
|B00KC812WK|2-pack OnePurify ...|
|B00MEPNYRK|GE PSB48YSHSS Pro...|
|B000JONLMQ|KitchenAid 18" TR...|
|B0073M7GNC|Whirlpool WOS92EC...|
|B00BIWR3IQ|GE Profile PWE23K...|
|B00O3XF1RC|MobileWasher Kit ...|
|B00JDB6P3I|Bosch NGM5055UC 5...|
|B0011YJE7Y|GE PHP960DMBB Pro...|
|B00J4EBYBM|Bosch HBL5651UC 5...|
|B0056HJ07Q|Frigidaire FHPC36...|
|B00FGWW82A|Frigidaire FFFS51...|
|B004UM5Y32|None Ers30t10074 ...|
|B00MANTPJM|GE Profile PFE28R...|
|B006L8PW1C|Fisher Paykel DD2...|
|B00T9WOH5E|Lg Lt600p Compati...|
|B001DHRJSU|Whirlpool Part Nu...|
|B0050KKN8E|Whirlpool Part Nu...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
products_df.count()

11694

In [ ]:
cus_count = dropna_df.groupBy('customer_id').count().orderBy('customer_id')
cus_count.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|      10097|    1|
|      10150|    1|
|      10947|    1|
|      11007|    1|
|      11804|    1|
|      11809|    1|
|      12064|    1|
|      12404|    1|
|      13112|    1|
|      13226|    1|
|      14408|    1|
|      14864|    1|
|      14889|    1|
|      14977|    1|
|      15033|    1|
|      15293|    1|
|      15715|    1|
|      16164|    1|
|      18873|    2|
|      18899|    1|
+-----------+-----+
only showing top 20 rows



In [ ]:
customers_df = cus_count.withColumnRenamed("count", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|      10097|             1|
|      10150|             1|
|      10947|             1|
|      11007|             1|
|      11804|             1|
|      11809|             1|
|      12064|             1|
|      12404|             1|
|      13112|             1|
|      13226|             1|
|      14408|             1|
|      14864|             1|
|      14889|             1|
|      14977|             1|
|      15033|             1|
|      15293|             1|
|      15715|             1|
|      16164|             1|
|      18873|             2|
|      18899|             1|
+-----------+--------------+
only showing top 20 rows



In [ ]:
customers_df.count()

91299

In [ ]:
# Create vine df for SQL table
vine_df = dropna_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R203HPW78Z7N4K|          5|            0|          0|   N|
|R2EAIGVLEALSP3|          5|            1|          1|   N|
|R1K1CD73HHLILA|          5|            0|          0|   N|
|R2KZBMOFRMYOPO|          5|            1|          1|   N|
| R6BIZOZY6UD01|          5|            0|          0|   N|
|R1MCXZFNF8E7Y0|          1|            0|          0|   N|
|R3EMB3E3ODR6BW|          5|            2|          2|   N|
| RJTONVTTOPJ5S|          5|            0|          0|   N|
|R21U5QZ2CQECUM|          4|            0|          0|   N|
| RL2BBC51H89DH|          4|            0|          0|   N|
|R3RNEPHF3WIRSZ|          2|            0|          0|   N|
|R38DNT9KML2PF3|          5|            0|          0|   N|
|R2ECMBJM8KNNC8|          4|            0|          0|   N|
|R2F3F92PRN9T7S|          5|            

In [ ]:
vine_df.count()

96888

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://amazonhwdb.canocujdsejy.us-east-2.rds.amazonaws.com:5432/amazonhwDB"
config = {"user":"postgres", 
          "password": "postgres", 
          "driver":"org.postgresql.Driver"}

In [ ]:
review_id_table_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)